In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
# Store csv created in part one into a DataFrame
#city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df = pd.read_csv("../WeatherPy/output/weather_data.csv")

city_data_df.head()

,Unnamed: 0,City,Country,Latitude,longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,0,Togur,RU,58.36,82.83,28.63,95,99,8.37
1,1,Cape Town,ZA,-33.93,18.42,66.20,45,0,28.86
2,2,Bluff,NZ,-46.60,168.33,46.99,92,7,4.65
3,3,Vardø,NO,70.37,31.11,33.80,80,75,19.46
4,4,Puerto Baquerizo Moreno,EC,-0.90,-89.60,66.49,80,99,11.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
print(city_data_df.count())
# Configure gmaps
gmaps.configure(api_key=g_key)

Unnamed: 0          1149
City                1149
Country             1140
Latitude            1149
longitude           1149
Temperature (F)     1149
Humidity (%)        1149
Cloudiness (%)      1149
Wind Speed (MPH)    1149
dtype: int64


In [34]:
# Heatmap of humidity
locations = city_data_df[["Latitude", "longitude"]]
humidity = city_data_df["Humidity (%)"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Temperature (F)"] < 80) & (city_data_df["Temperature (F)"] > 70) \
                                    & (city_data_df["Wind Speed (MPH)"] < 10) \
                                    & (city_data_df["Cloudiness (%)"] == 0)].dropna()
narrowed_city_df

,Unnamed: 0,City,Country,Latitude,longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
121,121,Hermanus,ZA,-34.42,19.23,71.01,48,0,8.99
259,259,Hermanus,ZA,-34.42,19.23,71.01,48,0,8.99
302,302,Hermanus,ZA,-34.42,19.23,71.01,48,0,8.99
313,313,Hermanus,ZA,-34.42,19.23,71.01,48,0,8.99
328,328,Hermanus,ZA,-34.42,19.23,71.01,48,0,8.99
429,429,Pochutla,MX,15.74,-96.47,73.99,93,0,3.24
451,451,Kādīpur,IN,26.17,82.38,74.44,33,0,3.58
462,462,Tezu,IN,27.92,96.17,70.66,56,0,3.58
501,501,Hermanus,ZA,-34.42,19.23,71.01,48,0,8.99
534,534,Abu Samrah,SY,35.30,37.18,70.14,45,0,7.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Latitude", "longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,longitude,Hotel Name
121,Hermanus,ZA,-34.42,19.23,
259,Hermanus,ZA,-34.42,19.23,
302,Hermanus,ZA,-34.42,19.23,
313,Hermanus,ZA,-34.42,19.23,
328,Hermanus,ZA,-34.42,19.23,
429,Pochutla,MX,15.74,-96.47,
451,Kādīpur,IN,26.17,82.38,
462,Tezu,IN,27.92,96.17,
501,Hermanus,ZA,-34.42,19.23,
534,Abu Samrah,SY,35.30,37.18,


In [38]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["longitude"]
    lng = row["longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Latitude,longitude,Hotel Name
121,Hermanus,ZA,-34.42,19.23,
259,Hermanus,ZA,-34.42,19.23,
302,Hermanus,ZA,-34.42,19.23,
313,Hermanus,ZA,-34.42,19.23,
328,Hermanus,ZA,-34.42,19.23,
429,Pochutla,MX,15.74,-96.47,
451,Kādīpur,IN,26.17,82.38,
462,Tezu,IN,27.92,96.17,
501,Hermanus,ZA,-34.42,19.23,
534,Abu Samrah,SY,35.30,37.18,Yeşilce market baklava ve fıstık maraş dondurm...


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "longitude"]]

In [40]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))